## DATA VISUALIZATION TEAM - Data Cleaning and Exploration

The purpose of this notebook is to narrow down the dataframe to useful information for the visualizations and explore data.

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [87]:
url = "https://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv"
rdf = pd.read_csv(url, parse_dates=["INCDTTM"])

In [88]:
rdf.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.362907,47.558372,1,326120,327620,3824232,Unmatched,Block,NaN,DELRIDGE WAY SW BETWEEN SW EDMUNDS ST AND SW H...,...,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
1,-122.301868,47.618931,2,65200,65200,2621299,Matched,Block,NaN,E DENNY WAY BETWEEN 23RD AVE AND 24TH AVE,...,Ice,Dusk,NaN,7015014.0,NaN,32,One parked--one moving,0,0,N
2,-122.391508,47.669461,3,20100,20100,1473746,Matched,Block,NaN,NW 56TH ST BETWEEN 26TH AVE NW AND 28TH AVE NW,...,Dry,Dark - Street Lights On,NaN,4080013.0,NaN,32,One parked--one moving,0,0,N
3,-122.346296,47.620903,4,49300,49300,2072241,Matched,Intersection,28927.0,TAYLOR AVE N AND THOMAS ST,...,Dry,Daylight,NaN,6154013.0,NaN,10,Entering at angle,0,0,N
4,-122.288688,47.532714,5,29800,29800,1177964,Unmatched,Block,NaN,BEACON ER AVE S BETWEEN S PORTLAND ST AND S CH...,...,NaN,NaN,NaN,4315006.0,NaN,NaN,NaN,0,0,N


#### Need to decide how to deal with columns with high percentage of missing values

In [89]:
col_na = round(100*(rdf.isna().sum()/len(rdf)), 2)
col_nunique = rdf.nunique()
df_summary = pd.DataFrame({"data_type": rdf.dtypes,
                           "percent_missing_values": col_na,
                           "total_unique_values": col_nunique}).sort_values(by=["percent_missing_values"],
                                                                            ascending=False)
df_summary.head(25)

,data_type,percent_missing_values,total_unique_values
PEDROWNOTGRNT,object,97.65,1
SPEEDING,object,95.52,1
EXCEPTRSNDESC,object,94.67,1
INATTENTIONIND,object,86.30,1
INTKEY,float64,67.56,7894
EXCEPTRSNCODE,object,54.64,2
SDOTCOLNUM,float64,42.27,127186
LIGHTCOND,object,11.99,9
WEATHER,object,11.95,11
ROADCOND,object,11.92,9


In [90]:
rdf['PEDROWNOTGRNT'].value_counts()

Y    5177
Name: PEDROWNOTGRNT, dtype: int64

Column that states whether or not the pedestrian right of way was *not* granted. (Y/N)

In all other cases, indicates that pedestrian right of way was granted. But it only makes sense to impute 'No' when pedestrian were involved in the accident. Otherwise, perhaps grouping remainder of missing values into category 'No Pedestrians' ???

In [91]:
rdf['SPEEDING'].value_counts()

Y    9870
Name: SPEEDING, dtype: int64

Reasonable to assume that when 'SPEEDING' column is null, individual wasn't speeding. Impute 'N'.

In [92]:
rdf['SPEEDING'].fillna('N', inplace = True)

In [94]:
rdf['SPEEDING'].head()

0    N
1    N
2    N
3    N
4    N
Name: SPEEDING, dtype: object

In [95]:
rdf.columns

Index(['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS',
       'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES',
       'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE',
       'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

Dropping not relevant columns

In [96]:
rdf.drop(['OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'EXCEPTRSNCODE', 'SDOT_COLCODE', 'SDOTCOLNUM'],axis=1,inplace = True)

In [99]:
rdf.columns

Index(['X', 'Y', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'SEVERITYCODE',
       'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
       'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES',
       'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLDESC', 'INATTENTIONIND',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT',
       'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR'],
      dtype='object')

Are we renaming column names?

In [39]:
rdf['SEVERITYDESC'].value_counts()

Property Damage Only Collision    136964
Injury Collision                   58441
Unknown                            21510
Serious Injury Collision            3077
Fatality Collision                   346
Name: SEVERITYDESC, dtype: int64

In [43]:
new_df = rdf[rdf['SEVERITYDESC']=='Serious Injury Collision']

#### Injury count includes Serious Injury count

In [45]:
new_df[['SEVERITYDESC','INJURIES','SERIOUSINJURIES']].head()

,SEVERITYDESC,INJURIES,SERIOUSINJURIES
25,Serious Injury Collision,1,1
146,Serious Injury Collision,1,1
239,Serious Injury Collision,2,1
280,Serious Injury Collision,1,1
451,Serious Injury Collision,1,1


In [8]:
conda install basemap

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - basemap


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    basemap-1.2.0              |   py37h0acbc05_0        14.4 MB
    conda-4.8.3                |           py37_0         2.8 MB
    geos-3.6.2                 |       h5470d99_2         1.1 MB
    proj4-5.2.0                |       h0a44026_1         6.6 MB
    pyproj-1.9.6               |   py37h9c430a6_0          58 KB
    pyshp-2.1.0                |             py_0          35 KB
    ------------------------------------------------------------
                                           Total:        25.0 MB

The following 

In [20]:
pip install geopandas

     |████████████████████████████████| 962kB 4.0MB/s eta 0:00:01
     |████████████████████████████████| 13.9MB 3.2MB/s eta 0:00:01     |█████████████████████████▊      | 11.2MB 2.7MB/s eta 0:00:02
     |████████████████████████████████| 1.6MB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 13.0MB 2.9MB/s eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
Note: you may need to restart the kernel to use updated packages.
